In [285]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [286]:
df_train = pd.read_csv('./train.csv')

In [287]:
#Extend the df by splitting the datetime into new categories
df_train['year'] = df_train.datetime.apply(lambda x: x.split()[0].split('-')[0])
df_train['month'] = df_train.datetime.apply(lambda x: x.split()[0].split('-')[1])
df_train['day'] = df_train.datetime.apply(lambda x: x.split()[0].split('-')[2])
df_train['hour'] = df_train.datetime.apply(lambda x : x.split()[1].split(":")[0])

#Infer Day Name
sr = pd.to_datetime(df_train['datetime']) 
df_train['Day_name'] = sr.dt.day_name()


#Season List 
season_dict = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}
df_train['Season_Exp'] = df_train['season'].map(season_dict)

#Weather List
weather_dict = {1:'clear' , 2:'mist' , 3:'light_precip' , 4: 'heavy_precip' }
df_train['Weather_Exp'] = df_train['weather'].map(weather_dict)

#Time List


In [288]:
df_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,Day_name,Season_Exp,Weather_Exp
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,01,01,00,Saturday,Spring,clear
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,01,01,01,Saturday,Spring,clear
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,01,01,02,Saturday,Spring,clear
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,01,01,03,Saturday,Spring,clear
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,01,01,04,Saturday,Spring,clear


In [289]:
#Night = 00 - 05
#Morning = 06-09
#Day = 10-16
#Evening =17-23

#night = np.sum(df_train.groupby('hour')['count'].sum()[0:5])
#morning = np.sum(df_train.groupby('hour')['count'].sum()[6:9])
#day = np.sum(df_train.groupby('hour')['count'].sum()[10:16])
#evening = np.sum(df_train.groupby('hour')['count'].sum()[17:23])

NameError: name 'np' is not defined

## Target Encode a Subset of Columns ##

In [290]:
all_columns = list(df_train.columns)

In [291]:
all_columns

['datetime',
 'season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'humidity',
 'windspeed',
 'casual',
 'registered',
 'count',
 'year',
 'month',
 'day',
 'hour',
 'Day_name',
 'Season_Exp',
 'Weather_Exp']

In [292]:
df_te_train = df_train

In [293]:
df_te_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,Day_name,Season_Exp,Weather_Exp
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,01,01,00,Saturday,Spring,clear
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,01,01,01,Saturday,Spring,clear
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,01,01,02,Saturday,Spring,clear
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,01,01,03,Saturday,Spring,clear
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,01,01,04,Saturday,Spring,clear


In [332]:
#Target Encoding a Subset of columns with MEAN


#Subset of large var list
column_list_to_target_encode = ['Season_Exp', 'Weather_Exp', 'year', 'month', 'Day_name', 'hour']
for column in column_list_to_target_encode:
    target_means = df_te_train.groupby(column).mean()
    df_te_train[f'{column}_te'] = df_te_train[column].replace(target_means['count'])



In [333]:
#List Conventions to define a training dataset

#for Categoricals
te = '_te'
te_list = [x + te for x in column_list_to_target_encode]
te_list

#num_list = ['temp', 'atemp', 'humidity', 'windspeed']
#num_list_3 = ['holiday','workingday', 'atemp', 'windspeed' ]
#te_list.extend(num_list)
te_list
#num_list_3

['Season_Exp_te',
 'Weather_Exp_te',
 'year_te',
 'month_te',
 'Day_name_te',
 'hour_te']

In [334]:
#Define Xtrain and ytrain
Xtrain = df_te_train[te_list]
#Xtrain = Xsub.values.reshape(-1,1)
ytrain = df_te_train['count']

In [335]:
Xtrain

,Season_Exp_te,Weather_Exp_te,year_te,month_te,Day_name_te,hour_te
0,116.343261,205.236791,144.223349,90.366516,196.665404,55.138462
1,116.343261,205.236791,144.223349,90.366516,196.665404,33.859031
2,116.343261,205.236791,144.223349,90.366516,196.665404,22.899554
3,116.343261,205.236791,144.223349,90.366516,196.665404,11.757506
4,116.343261,205.236791,144.223349,90.366516,196.665404,6.407240
5,116.343261,178.955540,144.223349,90.366516,196.665404,19.767699
6,116.343261,205.236791,144.223349,90.366516,196.665404,76.259341
7,116.343261,205.236791,144.223349,90.366516,196.665404,213.116484
8,116.343261,205.236791,144.223349,90.366516,196.665404,362.769231
9,116.343261,205.236791,144.223349,90.366516,196.665404,221.780220


In [336]:
#Scikit Linear Reg
m = LinearRegression()


In [337]:
m.fit(Xtrain, ytrain), m.score(Xtrain, ytrain)

(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False), 0.6819399746777768)

In [338]:
ypred = m.predict(Xtrain)
mean_squared_error(ytrain, ypred) #Scikit

10435.644494585393

In [339]:
m.coef_ , m.intercept_

(array([0.01746246, 0.96827038, 1.01053979, 1.00791012, 1.06947884,
        1.00640173]), -781.6345896562943)

In [340]:
te_list

['Season_Exp_te',
 'Weather_Exp_te',
 'year_te',
 'month_te',
 'Day_name_te',
 'hour_te']